In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('simData48Clean.csv')
df.head()

,date,event,fighter1,fighter1 - open,fighter1 - 48h,fighter1 - close,fighter2,fighter2 - open,fighter2 - 48h,fighter2 - close,winner
0,2025-03-09,UFC,Curtis Blaydes,2025-01-18 02:57:00 - price: -275,2025-03-06 15:01:00 - price: -339,2025-03-08 15:01:00 - price: -335,Rizvan Kuniev,2025-01-18 02:57:00 - price: +210,2025-03-06 15:01:00 - price: +259,2025-03-08 15:01:00 - price: +261,NaN
1,2025-03-09,UFC,Brunno Ferreira,2025-02-24 18:07:00 - price: +105,2025-03-06 21:01:00 - price: +116,2025-03-08 21:01:00 - price: +112,Armen Petrosyan,2025-02-24 18:07:00 - price: -125,2025-03-06 21:01:00 - price: -141,2025-03-08 21:01:00 - price: -134,Brunno Ferreira
2,2025-03-09,UFC,Rafael Fiziev,2025-02-27 05:11:00 - price: -139,2025-03-07 01:01:00 - price: -163,2025-03-08 23:01:00 - price: -159,Justin Gaethje,2025-02-27 05:11:00 - price: +112,2025-03-07 01:01:00 - price: +133,2025-03-08 23:01:00 - price: +131,Justin Gaethje
3,2025-03-09,UFC,Francis Marshall,2025-02-24 18:15:00 - price: +290,2025-03-06 17:01:00 - price: +247,2025-03-08 19:01:00 - price: +197,Mairon Santos,2025-02-24 18:15:00 - price: -370,2025-03-06 17:01:00 - price: -322,2025-03-08 19:01:00 - price: -245,Mairon Santos
4,2025-03-09,UFC,John Castaneda,2025-03-03 09:01:00 - price: -112,2025-03-06 15:01:00 - price: -106,2025-03-08 15:01:00 - price: -108,Chris Gutierrez,2025-03-03 09:01:00 - price: -112,2025-03-06 15:01:00 - price: -116,2025-03-08 15:01:00 - price: -112,NaN


In [3]:
print(df.isna().any(axis=1).sum())
df = df.dropna().reset_index(drop=True)
print(df.isna().any(axis=1).sum())

285
0


In [4]:
def calculate_profit(wager, price):
    if price > 0:
        profit = (price/100)*wager
    else:
        profit = (100/abs(price))*wager
    return profit


In [5]:
wager = 100
bankroll = wager * len(df)

profit_dict = {
    'favorite_at_open': bankroll,
    'favorite_at_close': bankroll,
    'favorite_at_48h': bankroll,

    'underdog_at_open': bankroll,
    'underdog_at_close': bankroll,
    'underdog_at_48h': bankroll,

    'favorite_random': bankroll,
    'underdog_random': bankroll,
    
    'true_random': bankroll
}
print(profit_dict)

{'favorite_at_open': 510300, 'favorite_at_close': 510300, 'favorite_at_48h': 510300, 'underdog_at_open': 510300, 'underdog_at_close': 510300, 'underdog_at_48h': 510300, 'favorite_random': 510300, 'underdog_random': 510300, 'true_random': 510300}


In [6]:
import random

for index, row in df.iterrows():
    fighter1 = row['fighter1']
    fighter2 = row['fighter2']
    winner = row['winner']

    #get open line
    open1 = int(row['fighter1 - open'].split('price: ')[1])
    open2 = int(row['fighter2 - open'].split('price: ')[1])

    close1 = int(row['fighter1 - close'].split('price: ')[1])
    close2 = int(row['fighter2 - close'].split('price: ')[1])

    forty81 = int(row['fighter1 - 48h'].split('price: ')[1])
    forty82 = int(row['fighter2 - 48h'].split('price: ')[1])

    lines = {
        fighter1: [open1, forty81, close1],
        fighter2: [open2, forty82, close2]
    }

    for_random = [open1, open2, forty81, close1, forty82, close2]

    neg_lines = []
    pos_lines = []
    for line in for_random:
        if(line > 0):
            pos_lines.append(line)
        else:
            neg_lines.append(line)

    if(len(pos_lines) >0):
        #dog random blindly takes a line
        random_dog = random.choice(pos_lines)
    else:
        #no pos line default to closest to pos
        random_dog = max(for_random)

    if(len(neg_lines)> 0):
        #fav random blindly takes a line
        random_fav = random.choice(neg_lines)
    else:
        random_fav = min(for_random)

    line_list = lines[winner]

    #if random fav picked a winning line
    if random_fav in line_list:
        profit_dict['favorite_random']+=calculate_profit(wager, random_fav)
    else:
        profit_dict['favorite_random']-=wager

    #if random dog picked a winning line
    if random_dog in line_list:
        profit_dict['underdog_random']+=calculate_profit(wager, random_dog)
    else:
        profit_dict['underdog_random']-=wager



    #true random
    random_fighter = row[f'fighter{random.randint(1,2)}']
    random_line = random.randint(0,1)
    true_random = lines[random_fighter][random_line]
    

    #if true random picked a winning line
    if true_random in line_list:
        profit_dict['true_random']+=calculate_profit(wager, true_random)
    else:
        profit_dict['true_random']-=wager

    #open fav line wins
    if(line_list[0] < 0):
        print(line_list[0])
        #favorite open bettors win
        profit_dict['favorite_at_open']+=calculate_profit(wager, line_list[0])
        #dog open bettor loses
        profit_dict['underdog_at_open']-=wager
    #open fav line loses
    else:
        profit_dict['favorite_at_open']-=wager
        profit_dict['underdog_at_open']+=calculate_profit(wager, line_list[0])

    #48h fav wins
    if(line_list[1] < 0):
        #favorite 48h bettors win
        profit_dict['favorite_at_48h']+=calculate_profit(wager, line_list[1])
        #dog 48h bettor loses
        profit_dict['underdog_at_48h']-=wager
    #48h fav line loses
    else:
        profit_dict['favorite_at_48h']-=wager
        profit_dict['underdog_at_48h']+=calculate_profit(wager, line_list[1])


    if(line_list[2] < 0):
        #favorite open bettors win
        profit_dict['favorite_at_close']+=calculate_profit(wager, line_list[2])
        #dog open bettor loses
        profit_dict['underdog_at_close']-=wager
    #open fav line loses
    else:
        profit_dict['favorite_at_close']-=wager
        profit_dict['underdog_at_close']+=calculate_profit(wager, line_list[2])



    
print(profit_dict)




-370
-600
-190
-345
-198
-110
-215
-549
-310
-130
-238
-148
-170
-310
-380
-230
-198
-186
-130
-225
-157
-192
-375
-455
-105
-120
-145
-230
-600
-230
-225
-250
-355
-425
-150
-250
-105
-350
-185
-200
-250
-500
-275
-290
-120
-230
-240
-235
-700
-500
-265
-112
-335
-375
-198
-700
-148
-205
-298
-142
-140
-1000
-130
-240
-600
-250
-180
-235
-420
-165
-120
-150
-275
-165
-130
-250
-195
-170
-250
-170
-278
-188
-752
-225
-155
-205
-170
-155
-218
-188
-148
-200
-350
-275
-258
-215
-175
-649
-150
-150
-110
-138
-549
-345
-649
-188
-107
-150
-120
-175
-163
-150
-110
-148
-305
-110
-250
-900
-145
-146
-198
-218
-148
-170
-115
-535
-1100
-285
-360
-245
-105
-155
-155
-145
-455
-350
-120
-278
-200
-350
-198
-105
-218
-200
-110
-230
-150
-175
-250
-120
-200
-142
-135
-155
-170
-105
-250
-278
-400
-455
-700
-115
-250
-218
-400
-110
-230
-105
-225
-125
-120
-549
-188
-163
-200
-163
-185
-285
-250
-200
-400
-275
-275
-250
-142
-185
-425
-250
-142
-395
-162
-225
-148
-198
-180
-156
-250
-170
-185
-13